# DataFrames Basics

## Prerrequisites

Install Spark and Java in VM

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [2]:
ls -l # check the .tgz is there

total 267680
drwxr-xr-x 1 root root      4096 Dec  7 14:41 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [3]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [4]:
!pip install -q findspark

In [5]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 200 kB 6.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define the environment

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [7]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [8]:
spark

In [9]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [11]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/cars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/movies.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset

In [12]:
ls -l /dataset

total 1784
-rw-r--r-- 1 root root  461474 Dec  9 17:01 bank.csv
-rw-r--r-- 1 root root   74910 Dec  9 17:00 cars.json
-rw-r--r-- 1 root root 1274347 Dec  9 17:01 movies.json
-rw-r--r-- 1 root root    4370 Dec  9 17:01 vehicles.csv


## Examples

Read from CSV to RDD and convert it to DF

In [13]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

In [14]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

In [15]:
bankText = spark.sparkContext.textFile("/dataset/bank.csv")

#we have to: remove firt row (headers), map the rest, and create DF
bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

bank.show(3)

+---+----------+-------+---------+-------+
|age|       job|marital|education|balance|
+---+----------+-------+---------+-------+
| 30|unemployed|married|  primary|   1787|
| 33|  services|married|secondary|   4789|
| 35|management| single| tertiary|   1350|
+---+----------+-------+---------+-------+
only showing top 3 rows



Read directly from JSON file to a DF

In [16]:
carsDF = spark.read.option("inferSchema", True).json("/dataset/cars.json") # inferSchema requires one extra pass over the data

# if None is set, it uses de default value (default = False) you can also pass the schema manually

Read directly from csv

In [17]:
bankDF = spark.read.option("header", "true").option("delimiter", ";").csv("/dataset/bank.csv")
bankDF.show(3)

+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|  services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
only showing top 3 rows



Showing a DF and print schema

In [ ]:
carsDF.show(2)
carsDF.printSchema()

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 2 rows

root
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- 

Get Rows

In [18]:
carsDF.take(2)

[Row(Acceleration=12.0, Cylinders=8, Displacement=307.0, Horsepower=130, Miles_per_Gallon=18.0, Name='chevrolet chevelle malibu', Origin='USA', Weight_in_lbs=3504, Year='1970-01-01'),
 Row(Acceleration=11.5, Cylinders=8, Displacement=350.0, Horsepower=165, Miles_per_Gallon=15.0, Name='buick skylark 320', Origin='USA', Weight_in_lbs=3693, Year='1970-01-01')]

Count

In [19]:
carsDF.count()

406

Schema

In [ ]:
# obtain a schema
carsSchema = carsDF.schema
print(type(carsSchema))
print(carsSchema)

<class 'pyspark.sql.types.StructType'>
StructType([StructField('Acceleration', DoubleType(), True), StructField('Cylinders', LongType(), True), StructField('Displacement', DoubleType(), True), StructField('Horsepower', LongType(), True), StructField('Miles_per_Gallon', DoubleType(), True), StructField('Name', StringType(), True), StructField('Origin', StringType(), True), StructField('Weight_in_lbs', LongType(), True), StructField('Year', StringType(), True)])


Custom Schemas

In [20]:
example = spark.sparkContext.parallelize([("chevrolet chevelle malibu",18,"1970-01-01","USA"),
    ("buick skylark 320",15,"1970-01-01","USA")])

In [ ]:
exampleDF = spark.createDataFrame(example)
exampleDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)



With columns names

In [ ]:
names = list(["name", "weight", "date", "country"])

In [ ]:
example2DF = example.toDF(names)
example2DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: long (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [21]:
# importing sql types
from pyspark.sql.types import *

In [ ]:
# custom schema
customSchema = StructType([ \
    StructField('name', StringType(), True), \
    StructField('weight', StringType(), True), \
    StructField('date', StringType(), True), \
    StructField('country', StringType(), True)])

In [ ]:
example3DF = spark.createDataFrame(example, customSchema)
example3DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
example3DF.show(2, False)

+-------------------------+------+----------+-------+
|name                     |weight|date      |country|
+-------------------------+------+----------+-------+
|chevrolet chevelle malibu|18    |1970-01-01|USA    |
|buick skylark 320        |15    |1970-01-01|USA    |
+-------------------------+------+----------+-------+



In [ ]:
# we can also specify schema with DDL (Data Definition Language)
customSchema2 = "`name` STRING NOT NULL, `weight` INT, `date` STRING, `country` STRING"

In [ ]:
example4DF = spark.createDataFrame(example, customSchema2)
example4DF.printSchema()

root
 |-- name: string (nullable = false)
 |-- weight: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
print(type(example2DF.collect()[0]["weight"]))
print(type(example3DF.collect()[0]["weight"]))

<class 'int'>
<class 'str'>


## Exercises
1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

3) Take a look to vehicles.csv. Read the file to a DF but this time with your own schema

Exercise 1

In [ ]:
smartphones = spark.sparkContext.parallelize([
    ("Samsung", "Galaxy S10", "Android", 12),
    ("Apple", "iPhone X", "iOS", 13),
    ("Nokia", "3310", "THE BEST", 0)])

In [ ]:
smartphonesDF = smartphones.toDF(["Make", "Model", "Platform", "CameraMegapixels"])

In [ ]:
smartphonesDF.show()

+-------+----------+--------+----------------+
|   Make|     Model|Platform|CameraMegapixels|
+-------+----------+--------+----------------+
|Samsung|Galaxy S10| Android|              12|
|  Apple|  iPhone X|     iOS|              13|
|  Nokia|      3310|THE BEST|               0|
+-------+----------+--------+----------------+



Exercise 2

In [ ]:
moviesDF = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load("/dataset/movies.json")

In [ ]:
moviesDF.printSchema()
print(f"The Movies DF has {moviesDF.count()} rows")

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

The Movies DF has 3201 rows


Exercise 3

In [ ]:
!head /dataset/vehicles.csv

name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class
Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled
T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft
X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NA,repulsorcraft
TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NA,6.4,1200,1,0,65,2 days,starfighter
Snowspeeder,t-47 airspeeder,Incom corporation,NA,4.5,650,2,0,10,none,airspeeder
TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NA,7.8,850,1,0,none,2 days,space/planetary bomber
AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,20,60,5,40,1000,NA,assault walker
AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,2,90,2,0,200,none,walker
Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin

In [ ]:
vehicles = spark.read.option("header", "true").option("inferSchema", "true").csv("/dataset/vehicles.csv")
vehicles.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: string (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: string (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



In [ ]:
vehiclesSchema = StructType([StructField('name', StringType(), True), StructField('model', StringType(), True), StructField('manufacturer', StringType(), True), 
                            StructField('cost_in_credits', LongType(), True), StructField('length', DoubleType(), True), StructField('max_atmosphering_speed', DoubleType(), True), 
                            StructField('crew', IntegerType(), True), StructField('passengers', IntegerType(), True), 
                            StructField('cargo_capacity', LongType(), True), StructField('consumables', StringType(), True), StructField('vehicle_class', StringType(), True)])

In [ ]:
vehiclesDF = spark.read.option("header", "true").schema(vehiclesSchema).csv("/dataset/vehicles.csv")
vehiclesDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: long (nullable = true)
 |-- length: double (nullable = true)
 |-- max_atmosphering_speed: double (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- cargo_capacity: long (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)

